In [1]:
# Optional: check if venv is activated

import sys

def is_venv():
    return (hasattr(sys, 'real_prefix') or
            (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix))

if is_venv():
    print('inside virtualenv or venv')
else:
    print('outside virtualenv or venv')

outside virtualenv or venv


In [2]:
import pandas as pd
import pandapipes as pp
import numpy as np

## Simulation variables and options

# Ambient temperature [K] = t[C] + 273.15
temp_ext_c = 20
temp_ext_k = temp_ext_c + 273.15

# Initial network fluid temperature (flow side) [K] = t[C] + 273.15
t_net_flow_init_c = 85
t_net_flow_init_k = t_net_flow_init_c + 273.15

# Initial network fluid temperature (return side) [K] = t[C] + 273.15
t_net_return_init_c = 35
t_net_return_init_k = t_net_return_init_c + 273.15

# Initial junction and network pressure, flow side [bar]
net_flow_p_bar = 10

# Initial junction and network pressure, return side [bar]
net_return_p_bar = 8

In [3]:
## Import data from Data.xls into Pandas dataframes

# Define data file path
sourcefile = './data/Data.xlsx'

# Import data
df_heater = pd.read_excel(sourcefile, sheet_name=0)
df_sink = pd.read_excel(sourcefile, sheet_name=1)
df_connection = pd.read_excel(sourcefile, sheet_name=2)
df_nodetype = pd.read_excel(sourcefile, sheet_name=3)

# Prepare blank network for elements
net = pp.create_empty_network(name="Data", fluid="water")

In [4]:
# for pipe in df_connection:
#     for node in df_nodetype:
#         if(str(node['Name'])==str(pipe['Start Node'])):
#             snt=node['Node Type']
#             break
#     for node in df_nodetype:
#         if(str(node['Name'])==str(pipe['End Node'])):
#             ent=node['Node Type']
#             break
#     print(f'{snt} - {pipe["Start Node"]} : {ent} - {pipe["End Node"]}')
#     print()

lenpipe=len(df_connection)
lennode=len(df_nodetype)
for i in range(lenpipe):
    sn_id=df_connection.at[i,'Start Node']
    en_id=df_connection.at[i,'End Node']
    for j in range(lennode):
        if df_nodetype.at[j,'Name']==sn_id:
            snt=df_nodetype.at[j,'Node Type']
            break
    for j in range(lennode):
        if df_nodetype.at[j,'Name']==en_id:
            ent=df_nodetype.at[j,'Node Type']
            break
    print(f'{snt} - {sn_id} : {ent} - {en_id}')
    print()
           

Junction - Junction-165 : Sink/User - Junction-192

Junction - Junction-1395 : Sink/User - Junction-736

Junction - Junction-897 : Sink/User - Junction-894

Junction - Junction-916 : Junction - Junction-909

Junction - Junction-919 : Junction - Junction-920

Junction - Junction-916 : Junction - Junction-919

Junction - Junction-942 : Sink/User - Junction-1164

Junction - Junction-954 : Sink/User - Junction-945

Junction - Junction-998 : Sink/User - Junction-956

Junction - Junction-998 : Junction - Junction-954

Junction - Junction-1036 : Junction - Junction-916

Junction - Junction-1036 : Junction - Junction-907

Junction - Junction-1143 : Sink/User - Junction-1124

Junction - Junction-1044 : Sink/User - Junction-1114

Junction - Junction-1395 : Junction - Junction-1396

Junction - Junction-1455 : Junction - Junction-1459

Junction - Junction-1459 : Junction - Junction-1457

Junction - Junction-1459 : Junction - Junction-1461

Junction - Junction-1471 : Junction - Junction-1470

Junct

In [5]:
lenpipe=len(df_connection)
lennode=len(df_nodetype)
sink,heat,junction = (0,)*3
for i in range(lennode):
    if str(df_nodetype.at[i,'Node Type'])=='Sink/User':
        sink+=1
    elif str(df_nodetype.at[i,'Node Type'])=='Source/Heater':
        heat+=1
    elif str(df_nodetype.at[i,'Node Type'])=='Junction':
        junction+=1
jj, ss, js, sj, hj, jh, hs, sh, hh = (0,) * 9
for i in range(lenpipe):
    sn_id=df_connection.at[i,'Start Node']
    en_id=df_connection.at[i,'End Node']
    for j in range(lennode):
        if df_nodetype.at[j,'Name']==sn_id:
            snt=df_nodetype.at[j,'Node Type']
            snt_id=df_nodetype.at[j,'Name']
            break
    for j in range(lennode):
        if df_nodetype.at[j,'Name']==en_id:
            ent=df_nodetype.at[j,'Node Type']
            ent_id=df_nodetype.at[j,'Name']
            break
    # print(f'{snt} - {sn_id} : {ent} - {en_id}')
    # print()
    if snt=='Junction' and ent=='Junction':
        jj+=1
    elif snt=='Sink/User' and ent=='Sink/User':
        ss+=1
    elif snt=='Junction' and ent=='Sink/User':
        js+=1
    elif snt=='Sink/User' and ent=='Junction':
        sj+=1
        #print(f'snt_id: {snt_id} and ent_id: {ent_id}')
    elif snt=='Source/Heater' and ent=='Junction':
        hj+=1
    elif ent=='Source/Heater' and snt=='Junction':
        jh+=1
    elif snt=='Source/Heater' and ent=='Sink/User':
        hs+=1
    elif ent=='Source/Heater' and snt=='Sink/User':
        sh+=1
    elif snt=='Source/Heater' and ent=='Source/Heater':
        hh+=1
print(f'Heat: {heat}\nJunction: {junction}\nSink: {sink}')
print(f'jj: {jj}\nss: {ss}\njs: {js}\nsj: {sj}\nhj: {hj}\njh: {jh}\nhs: {hs}\nsh: {sh}\nhh: {hh}')
    
    

Heat: 3
Junction: 794
Sink: 496
jj: 808
ss: 0
js: 486
sj: 13
hj: 1
jh: 3
hs: 0
sh: 0
hh: 0


In [6]:
## Create pandapipes junctions from imported data.
# Creates separate networks for supply and return lines. Supply and return
# lines join at sinks and heat sources.

# Get number of nodes from dataframe
num_nodes = len(df_nodetype)

# tracker tracks number of junctions created
junction_tracker = 0

# Iterate over all nodes to find and create junctions
for i in range(num_nodes):

    if df_nodetype.at[i, 'Node Type'] == "Junction":
        
        # Replace dashes with underscores to conform with Python markup
        junction_name = str(df_nodetype.at[i, 'Name']).replace("Junction-",
                                                               "Junction_")
        
        junction_pos_x = float(df_nodetype.at[i, 'X-Coordinate'])
        junction_pos_y = float(df_nodetype.at[i, 'Y-Coordinate'])
        junction_pos_flow = (junction_pos_x, junction_pos_y)
        #junction_pos_return = (junction_pos_x, junction_pos_y-100)

        # Define supply and return flow network junctions
        pp.create_junction(net,
            pn_bar=net_flow_p_bar,
            tfluid_k=t_net_flow_init_k,
            geodata=junction_pos_flow,
            name=junction_name + '_supply')
        # pp.create_junction(net,
        #     pn_bar=net_return_p_bar,
        #     tfluid_k=t_net_return_init_k,
        #     geodata=junction_pos_return,
        #     name=junction_name + '_return')

        junction_tracker += 1

print("Number of junctions created:", junction_tracker)

# List junctions
net.junction

Number of junctions created: 794


,name,pn_bar,tfluid_k,height_m,in_service,type
0,Junction_165_supply,10.0,358.15,0.0,True,junction
1,Junction_300_supply,10.0,358.15,0.0,True,junction
2,Junction_897_supply,10.0,358.15,0.0,True,junction
3,Junction_907_supply,10.0,358.15,0.0,True,junction
4,Junction_909_supply,10.0,358.15,0.0,True,junction
5,Junction_916_supply,10.0,358.15,0.0,True,junction
6,Junction_917_supply,10.0,358.15,0.0,True,junction
7,Junction_919_supply,10.0,358.15,0.0,True,junction
8,Junction_920_supply,10.0,358.15,0.0,True,junction
9,Junction_942_supply,10.0,358.15,0.0,True,junction


In [7]:
# Conversion factor for energy-to-mass flow
e_flow_conv_ratio = (7 / 0.033 + 40.3 / 0.193) / 2  # = 210.4648

# Temperature in Kelvin
t_out_k = 90 + 273.15

# Flow-side pressure
source_flow_bar = 10
pressure_lift = 3

# Production multiplier
prod_multiplier = 1

# Loop over heaters
for _, row in df_heater.iterrows():
    # Create junction and store the returned ID
    junction_id = pp.create_junction(
        net,
        pn_bar=source_flow_bar,
        tfluid_k=t_out_k,
        geodata=(float(df_nodetype.at[row.name, 'X-Coordinate']),
                 float(df_nodetype.at[row.name, 'Y-Coordinate'])),
        name=row['Name']
    )

    # Create source with dynamic mass flow
    pp.create_source(
        net,
        junction=junction_id,
        mdot_kg_per_s=190.0,
        in_service=True
    )
